In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
vic_data = pd.read_csv('victoria.csv', parse_dates=[0])

Feels like by grouping the prices using period of the day, the alogirthm is kind of fixed (so that it does not make decision using each day's price, it uses the overall price), seems like the method would be closer to algorithm 1. (the grouping is in another notebook named `chieh.ipynb`)

In [3]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1
TRADING_PRICE = 'Regions VIC Trading Price ($/MWh)'

In [4]:
CHARGING_LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25

DISCHARGING_LOOKAHEAD = 10
DISCHARGING_PERCENTILE = 0.75

In [5]:
vic_price = vic_data[['Time (UTC+10)',TRADING_PRICE]]

In [6]:
vic_price['charging_or_not'] = 0
vic_price['discharging_or_not'] = 0

vic_price['raw_power'] = 0

vic_price['market_dispatch'] = 0

vic_price['opening_capacity'] = 0
vic_price['closing_capacity'] = 0

vic_price['revenue'] = 0

In [7]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [8]:
import time

In [9]:
start_time_charge = time.time()

for i in list(vic_price.index)[1:]:

    if((i+CHARGING_LOOKAHEAD+1) < len(vic_price)):
        
        thelist = vic_price.iloc[i+1:i+CHARGING_LOOKAHEAD+1][TRADING_PRICE]
        ser = pd.Series(thelist)

        
        if(quantile_exc(ser,0.25) < vic_price.at[i,TRADING_PRICE]):
            vic_price.at[i,'charging_or_not'] = 0
        else:
            vic_price.at[i,'charging_or_not'] = 1
            
            
        if(quantile_exc(ser,0.75) > vic_price.at[i,TRADING_PRICE]):
            vic_price.at[i,'discharging_or_not'] = 0
        else:
            vic_price.at[i,'discharging_or_not'] = 1
            
            
end_time_charge = time.time()
behavior_time = end_time_charge - start_time_charge

In [10]:
start_time = time.time()


for i in list(vic_price.index)[1:]:
    
    
    # update opening capacity
    if(i != 0):
        vic_price.at[i, 'opening_capacity'] = vic_price.at[i-1, 'closing_capacity']
    
    
    # finding raw_power -- this uses the modified function Akira gave us in the last announcement
    if(vic_price.at[i,'charging_or_not'] == 1):
        vic_price.at[i,'raw_power'] = -1*min(BATTERY_POWER, 
                                           (BATTERY_CAPACITY-vic_price.at[i,'opening_capacity'])/EFFICIENCY*2)

    if(vic_price.at[i,'discharging_or_not'] == 1):
        vic_price.at[i,'raw_power'] = min(BATTERY_POWER, vic_price.at[i,'opening_capacity']/EFFICIENCY*2)
        
        
    # finding market_dispatch 
    if(vic_price.at[i,'raw_power'] < 0):
        vic_price.at[i,'market_dispatch'] = vic_price.at[i,'raw_power']/2
        
    else:
        # EFFICIENCY is already in decimal (0.9) so no need to divide by 100
        vic_price.at[i,'market_dispatch'] = vic_price.at[i,'raw_power']/2 * EFFICIENCY 
        
        
    # finidng closing_capacity   
    if(vic_price.at[i,'market_dispatch'] < 0):
        thecondition = vic_price.at[i,'opening_capacity'] - (vic_price.at[i,'market_dispatch'] * EFFICIENCY)
    else:
        thecondition = vic_price.at[i,'opening_capacity'] - (vic_price.at[i,'market_dispatch'] * (100/(EFFICIENCY*100)))
    
    vic_price.at[i,'closing_capacity'] = round(max(0, min(thecondition, BATTERY_CAPACITY)),0)
      
        
        
    #finding revenue        
    if(vic_price.at[i,'market_dispatch'] < 0):
        vic_price.at[i,'revenue'] = round(vic_price.at[i,'market_dispatch'] * vic_price.at[i,TRADING_PRICE] * (1/MARGINAL_LOSS_FACTOR),0)
    else:
        vic_price.at[i,'revenue'] = round(vic_price.at[i,'market_dispatch'] * vic_price.at[i,TRADING_PRICE] * MARGINAL_LOSS_FACTOR,0)


end_time = time.time()
loop_time = end_time - start_time

In [11]:
total_time_complexity = behavior_time + loop_time
print('Time complexity for algo1 is:', total_time_complexity, 'seconds')

Time complexity for algo1 is: 28.08889079093933 seconds


In [12]:
thetime = vic_price.loc[(vic_price['Time (UTC+10)'] >= '2020-07-17') & (vic_price['Time (UTC+10)'] < '2020-07-18')]

thetime

,Time (UTC+10),Regions VIC Trading Price ($/MWh),charging_or_not,discharging_or_not,raw_power,market_dispatch,opening_capacity,closing_capacity,revenue
44544,2020-07-17 00:00:00,79.09,0,1,0,0,0,0,0
44545,2020-07-17 00:30:00,75.51,0,1,0,0,0,0,0
44546,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,0
44547,2020-07-17 01:30:00,75.57,0,1,0,0,0,0,0
44548,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
44549,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,0
44550,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
44551,2020-07-17 03:30:00,58.04,1,0,-300,-150,0,135,-8785
44552,2020-07-17 04:00:00,51.85,1,0,-300,-150,135,270,-7848
44553,2020-07-17 04:30:00,74.53,0,0,0,0,270,270,0


In [13]:
sum(thetime['revenue'])

49239

In [ ]:
vic_price.to_csv('vic_price_algo3.csv')

In [14]:
print("Total Revenue:", vic_price['revenue'].sum())
print("Average Revenue (per day):", vic_price['revenue'].sum()/(len(vic_price)/48))

Total Revenue: 121360878
Average Revenue (per day): 91799.52005294924


- The total revenue here is higher than the total revenue in the given file, I think this is right cuz the file is only up to 2021/07/01, this dataset has more data than the one in the given example

- The dataset used contains all data (not only training but also the validation and testing part)

### Try using percentile.inc instead of percentile.exc excel equivalent function when calculating percentiles

### percentile.inc performed way worse than percentile.exc!

In [ ]:
# start_time_charge = time.time()

# for i in list(vic_price.index)[1:]:

#     if((i+CHARGING_LOOKAHEAD+1) < len(vic_price)):
        
#         thelist = vic_price.iloc[i+1:i+CHARGING_LOOKAHEAD+1][TRADING_PRICE]
        
#         if(np.percentile(thelist,0.25) < vic_price.at[i,TRADING_PRICE]):
#             vic_price.at[i,'charging_or_not'] = 0
#         else:
#             vic_price.at[i,'charging_or_not'] = 1
            
            
#         if(np.percentile(thelist,0.75) > vic_price.at[i,TRADING_PRICE]):
#             vic_price.at[i,'discharging_or_not'] = 0
#         else:
#             vic_price.at[i,'discharging_or_not'] = 1
            
            
# end_time_charge = time.time()
# behavior_time = end_time_charge - start_time_charge

In [ ]:
# start_time = time.time()

# for i in list(vic_price.index)[1:]:
    
    
#     # update opening capacity
#     if(i != 0):
#         vic_price.at[i, 'opening_capacity'] = vic_price.at[i-1, 'closing_capacity']
    
    
    
#     # finding raw_power -- this uses the modified function Akira gave us in the last announcement
#     if(vic_price.at[i,'charging_or_not'] == 1):
#         vic_price.at[i,'raw_power'] = -1*min(BATTERY_POWER, 
#                                            (BATTERY_CAPACITY-vic_price.at[i,'opening_capacity'])/EFFICIENCY*2)

#     if(vic_price.at[i,'discharging_or_not'] == 1):
#         vic_price.at[i,'raw_power'] = min(BATTERY_POWER, vic_price.at[i,'opening_capacity']/EFFICIENCY*2)
        
        
        
#     # finding market_dispatch 
#     if(vic_price.at[i,'raw_power'] < 0):
#         vic_price.at[i,'market_dispatch'] = vic_price.at[i,'raw_power']/2
        
#     else:
#         # EFFICIENCY is already in decimal (0.9) so no need to divide by 100
#         vic_price.at[i,'market_dispatch'] = vic_price.at[i,'raw_power']/2 * EFFICIENCY 
        
        
        
#     # finidng closing_capacity   
#     if(vic_price.at[i,'market_dispatch'] < 0):
#         thecondition = vic_price.at[i,'opening_capacity'] - (vic_price.at[i,'market_dispatch'] * EFFICIENCY)
#     else:
#         thecondition = vic_price.at[i,'opening_capacity'] - (vic_price.at[i,'market_dispatch'] * (100/(EFFICIENCY*100)))
    
#     vic_price.at[i,'closing_capacity'] = round(max(0, min(thecondition, BATTERY_CAPACITY)),0)
      
        
        
#     #finding revenue        
#     if(vic_price.at[i,'market_dispatch'] < 0):
#         vic_price.at[i,'revenue'] = round(vic_price.at[i,'market_dispatch'] * vic_price.at[i,TRADING_PRICE] * (1/MARGINAL_LOSS_FACTOR),0)
#     else:
#         vic_price.at[i,'revenue'] = round(vic_price.at[i,'market_dispatch'] * vic_price.at[i,TRADING_PRICE] * MARGINAL_LOSS_FACTOR,0)


# end_time = time.time()
# loop_time = end_time - start_time

In [ ]:
# total_time_complexity = behavior_time + loop_time
# print('Time complexity for algo1 is:', total_time_complexity, 'seconds')

In [ ]:
# print("Total Revenue:", vic_price['revenue'].sum())
# print("Average Revenue (per day):", vic_price['revenue'].sum()/(len(vic_price)/48))

In [ ]:
# sum(vic_price.loc[(vic_price['Time (UTC+10)'] >= '2020-07-17') & (vic_price['Time (UTC+10)'] < '2020-07-18')]['revenue'])

# # using np.percentile gives 24209 for the checkpoint, which is obvioulsy incorrect